# Intro to HPC: Cleaning and cannonizing DB

Project: Solar Power for Affordable Housing through Computational Design of Low-Cost/High-Efficiency Solar Cells.
Author: AlvaroVM [https://alvarovm.github.io](http://alvarovm.github.io)
Version: 0.0.1


The goal of this notebook is to clean  the file that containt the DyeDB generated from dumping a database of results from calculations and data minig literature. Given the complexity of the calculations, some of the cells are empty.

Using a Pandas `Dataframe` we will open the data set replace missing values with zeros, and add descriptors and fingerprints that would be useful to create molecular maps and apply machine learning.




In [ ]:
import sys
import os
SRC_DIR='..'

In [ ]:
sys.path.append(os.path.join(SRC_DIR, 'code'))
import utils

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
#https://github.com/jmcarpenter2/swifter
#import swifter
#2-TSNE-UMAP-map-cuda-Copy1

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import DataStructs 
from rdkit.Chem import Draw
from rdkit.Chem.rdMolDescriptors import  GetHashedMorganFingerprint
from rdkit.DataStructs import ConvertToNumpyArray

utils.plot_settings2()

results_path = os.path.join(SRC_DIR,'results')

### Tools to Cannonize smiles

In [ ]:
from rdkit.Chem import MolFromSmiles as smi2mol
from rdkit.Chem.rdMolDescriptors import  GetHashedMorganFingerprint,GetMorganFingerprintAsBitVect
from rdkit.DataStructs import ConvertToNumpyArray

import re

def canon_smiles(smi):
    try:
        m = smi2mol(smi)
    except:
        m = False
        print('hola cannon'+smi)
        
    if m is False:
        return False
    else:
        try:
            sim = Chem.MolToSmiles(m, isomericSmiles=True, canonical=True)
        except:
            sim = False
            #print('hola cannon2'+smi)
        return sim
    

def CleanSMI(smi):
        try: 
            clean=re.sub(r'[<>%\\/?\|]+', '', smi)
        except:
            #print('holaclean ' +smi )
            clean =False
        return clean

def HardValidSMI(smi):
    """
    A rule based function to validate a given smile string. 
    Return type: Boolean
    True: If a match is found. 
    False: Charges, Ions and No Conjugated regions found.
    """

    mysmile = CleanSMI(smi)

    if mysmile is not False:

        try: 
            illegalstring = re.search(r'\\|/|\*|Fe|\+\+|\.|\|',mysmile) #--> Sanity check!
        except:
            #print('holaill ' +smi )
            illegalstring = True
    else:
        return False
    
    if illegalstring:
        return False
    else:
        cansmile = canon_smiles(mysmile)
        if cansmile is False:
            return False
        match = re.search(r'\[\w{1,3}[\+-\.\d]+\]|\[\w{2}\]|\.|\(\*\)',cansmile)
        if match:
            return False
        else:
            conjuated = re.search(r'[a-z\W]\d+[a-zD-Z\W]+\d',mysmile) #r'[a-z\W]\d+[\w\W]+\d' or use (r'[a-z\W]\d+[a-zD-Z\W]+\d',mysmile) )
            if conjuated:
                return True
            else:
                return False


def applyMorganFP(m,**kwargs):
    fptype='bit'
    
    if 'fptype' in kwargs:
        fptype=kwargs['fptype']
    if 'fp_args' in kwargs:
        fp_args=kwargs['fp_args']     
    #fp_args = self.meta_data['fp_args']
    #fptype = self.meta_data['fptype']
    arr = np.zeros((1,))
    if fptype == 'bit': 
        arr = np.zeros((1,))
        #ConvertToNumpyArray(GetHashedMorganFingerprint(m, **fp_args), arr)
        try:
            arr = np.array(GetMorganFingerprintAsBitVect(m, **fp_args))
        except:
            print(Chem.MolToSmiles(m))
    elif fptype == 'count':
        #arr = np.zeros((1,))
        ConvertToNumpyArray(GetHashedMorganFingerprint(m, **fp_args), arr)
    return arr

## Open Database

Our database is expresed a CSV file (spread sheet like). This each row has information of the molecular description of in SMILE format, and optical properties as the wavelenght (lamnda/nm), extinction coefficient, first excitation energy with quantum mechanics, etc.

In [ ]:
df = pd.read_csv('../data/extended_db_Zindo_Nov_2019_V5.csv').fillna(value = 0)
print('Column names: {}'.format(str(df.columns.tolist())))
print('Table Shape: {}'.format(df.shape))

In [ ]:
#df.count()

Unfortunately some of the SMILE that define the molecules are not valid. We have a function called `HardValidSMI` which validate whether the SMILES can be processed. See example

In [ ]:
#smi=df['smi_post'].iloc[0]
smi = 'O=C1O[C@H]([C@@H](O)CO)C(O)=C1O'
print(f'Q: Is this {smi} a valid molecule? A: {HardValidSMI(smi)}')

Remove metals and invalid smiles. Adding a new column `nogood`. Notice that there two SMILES: `smile_pre` and `smile_post`. The `smile_pre` are the SMILES generated directly from the molecules found in the literature, and `smiles_post` are a reduced version with only the active region of the molecule.

In [ ]:
smi = 'Clc1ccc(c(c1)Cl)OCCn1cncc1'
print(f'Q: Is this {smi} a valid molecule? A: {canon_smiles(smi)}')

### Exercises:
* Apply the function `HardValidSMI` to the columns with SMILES. How many cells have `True` and `False` ?
* Create a `smiles` column with cannonized SMILES using the `canon_smiles` function.
* Keep only the rows with `lambda_sTDA (nm)` that are bigger than zero.
* Create column with a RDKIT `molecule` object using the SMILES.
* Create column with the Morgan fingerprint using the `applyMorganFP` to create a 2048 bit vectors using fragments up to radii 6, for example: `fp= applyMorganFP(mol,fptype='bit',fp_args={'radius':6, 'nBits':2048})`
* Compute a set of descriptors (see list below) and add them as extra columns to the Dataframe
* Compute the energy LUMO-HOMO gap from `mopac`, `zindo` and `dft`, compare them.
* Save the `Dataframe` as a pickle file.

#### Remove systems without sTDA (disabled)

In [ ]:
#df['nostda']=df['lambda_sTDA (nm)'].apply(np.isnan)
#df = df[df.nostda == False]
##df = df[df['lambda_sTDA (nm)'] >0 ]
##df.shape

###  Descritors list

In [ ]:
import inspect
import rdkit.Chem.Descriptors as Descriptors
#https://github.com/jmcarpenter2/swifter
#import swifter 
from collections import OrderedDict

getonly=['NHOHCount', 'NOCount', 'NumAliphaticCarbocycles',
                           'NumAliphaticHeterocycles', 'NumAliphaticRings',
                           'NumAromaticCarbocycles', 'NumAromaticHeterocycles',
                           'NumAromaticRings', 'NumHAcceptors', 'NumHDonors', 
                           'NumHeteroatoms', 'NumRadicalElectrons', 'NumRotatableBonds',
                           'NumSaturatedCarbocycles', 'NumSaturatedHeterocycles', 
                           'NumSaturatedRings', 'NumValenceElectrons',
                            ]

calc_props = OrderedDict(inspect.getmembers(Descriptors, inspect.isfunction))
for key in list(calc_props.keys()):
    if key.startswith('_'):
        del calc_props[key]
    else:
        thisnot=False
        for myprop in getonly:
            if myprop == key:
                thisnot=True
        if not thisnot:
            del calc_props[key]
            
from tqdm import tqdm
def calc_all(dfc,calc_props,smiles_col='smiles'):
    #df['mol'] = df[smiles_col].apply(Chem.MolFromSmiles)
    for key,val in tqdm (calc_props.items()):
    #for key,val in calc_props.items():
        #df[key] = df['mol'].apply(val)
        df[key] = df['mol'].apply(val)
    return df
print('Found {} molecular descriptors in RDKIT'.format(len(calc_props)))
#calc_props
#df=calc_all(df,calc_props)